# Grid SST Workflow

Execute the entire Grid SST workflow: input -> pre-process -> gp-regression -> ?

In [2]:
# Standard imports
from pathlib import Path

# Local imports
from ipynb.fs.full.preprocess import run_preprocessing

In [3]:
# Constants
OUT = Path("/home/jovyan/shared-readwrite/workflow_data/preprocess")

In [4]:
# VIIRS L2P Sensor data
short_name = 'VIIRS_NPP-STAR-L2P-v2.80'
provider = 'POCLOUD'
temporal_range = '2022-07-18T00:00:00Z,2022-07-24T23:59:59Z'
bbox = "-40,0,-35,5"
qc_threshold = 4

In [5]:
run_preprocessing(short_name, provider, temporal_range, bbox, qc_threshold, OUT)

100%|██████████| 31/31 [05:16<00:00, 10.22s/it]
